# **Setup**

In [3]:
#@title initialize paths
import os

home_path = os.path.join(os.getcwd(), "content\\FollowYourPose")
os.makedirs(home_path, exist_ok=True)
data_path = f"{home_path}\\data"
checkpoint_path = f"{home_path}/checkpoints"

In [7]:
#@title Install requirements

%pip install -q diffusers[torch]==0.11.1 transformers==4.26.0 bitsandbytes==0.35.4 \
decord accelerate omegaconf einops ftfy gradio imageio-ffmpeg xformers

!pip install av

!apt-get install imagemagick

!pip uninstall torch torchvision triton -y
!pip install torch==2.0.0 torchvision -f https://download.pytorch.org/whl/cu118/touch_stable.html
!pip install xformers==0.0.19 triton==2.0.0 -U
!pip uninstall nvidia-cudnn-cu11 -y
!pip install nvidia-cudnn-cu11==8.6.0.163

c:\Users\there\Desktop\SIT\Year3 Tri1\3104 Software Management\Project\project\ICT3104Project\content\FollowYourPose


Cloning into '.'...


Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


Found existing installation: torch 2.0.1
Uninstalling torch-2.0.1:
  Successfully uninstalled torch-2.0.1
Found existing installation: torchvision 0.15.1
Uninstalling torchvision-0.15.1:
  Successfully uninstalled torchvision-0.15.1


Looking in links: https://download.pytorch.org/whl/cu118/touch_stable.html
  Using cached torch-2.0.0-cp311-cp311-win_amd64.whl (172.3 MB)
  Using cached torchvision-0.16.0-cp311-cp311-win_amd64.whl (1.3 MB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
  Using cached torchvision-0.15.2-cp311-cp311-win_amd64.whl (1.2 MB)
  Using cached torchvision-0.15.1-cp311-cp311-win_amd64.whl (1.2 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.21 requires torch==2.0.1, but you have torch 2.0.0 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement xformers==0.0.19 (from versions: 0.0.1, 0.0.2, 0.0.3, 0.0.4, 0.0.5, 0.0.6, 0.0.7, 0.0.8, 0.0.9, 0.0.10, 0.0.11, 0.0.12, 0.0.13, 0.0.16rc424, 0.0.16rc425, 0.0.16, 0.0.20, 0.0.21, 0.0.22, 0.0.22.post4, 0.0.22.post7, 0.0.23.dev660, 0.0.23.dev661, 0.0.23.dev662, 0.0.23.dev670, 0.0.23.dev672)
ERROR: No matching distribution found for xformers==0.0.19

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement nvidia-cud

# **Data Exploration**

In [1]:
#@title Upload charades video into Data folder

#Change directory into data folder
%cd $data_path

import os
import shutil
from google.colab import files

# Create a function to upload and organize files
def upload_and_organize_files():
    # Upload the files
    uploaded = files.upload()

    for file_name, file_content in uploaded.items():
        # Extract the file name without the extension
        file_name_without_extension = os.path.splitext(file_name)[0]

        # Create a directory with the same name as the file
        file_directory = os.path.join(data_path, file_name_without_extension)
        os.makedirs(file_directory, exist_ok=True)

        # Save the file in its respective directory
        file_path = os.path.join(file_directory, file_name)
        with open(file_path, 'wb') as f:
            f.write(file_content)

        # Use shutil.move to move the file to the target directory
        shutil.move(file_name, os.path.join(file_directory, file_name))

        print(f'Saved file {file_name} in directory {file_name_without_extension}')

# Call the function to upload and organize files
upload_and_organize_files()

[WinError 2] The system cannot find the file specified: '$data_path'
c:\Users\there\Desktop\SIT\Year3 Tri1\3104 Software Management\Project\project\ICT3104Project


ModuleNotFoundError: No module named 'google'

In [ ]:
#@title Create Dropdown class

import os
import ipywidgets as widgets
from ipywidgets import Video, HBox
from IPython.display import display, clear_output

# A dropdown class to get all folders and videos within each folder
class FolderDropdown:
    def __init__(self, data_path, toDisplayVideo=False):
        self.data_path = data_path
        self.folder_names = [f for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))]
        self.dropdown = widgets.Dropdown(
            options=['--Select a folder--'] + self.folder_names,
            description='Select a folder:'
        )
        self.video_dropdown = widgets.Dropdown(
            options=['--Select a video--'],
            description='Select a video:'
        )
        self.output = widgets.Output()
        self.dropdown.observe(self.on_dropdown_change)
        self.selected_folder = '--Select a folder--'
        self.selected_video = '--Select a video--'
        self.toDisplay = toDisplayVideo

    def display(self):
        display(self.dropdown)
        display(self.video_dropdown)
        display(self.output)

    def on_dropdown_change(self, change):
        with self.output:
            self.selected_folder = self.dropdown.get_interact_value()
            clear_output(wait=True)

            if change['type'] == 'change' and change['name'] == 'value' and not str(change['new']) == "--Select a folder--":
                selected_folder = str(change['new'])
                folder_path = os.path.join(self.data_path, selected_folder)
                condition = lambda file_name: not file_name.startswith(".ipynb")
                video_files = [f for f in os.listdir(folder_path) if condition(f)]

                self.video_dropdown.options = ['--Select a video--'] + video_files
                self.video_dropdown.observe(self.on_video_dropdown_change)

            elif str(change['new']) == "--Select a folder--":
              self.video_dropdown.options = ['--Select a video--']

    def on_video_dropdown_change(self, change):
        with self.output:
          self.selected_video = self.video_dropdown.get_interact_value()

          if change['type'] == 'change' and change['name'] == 'value' and not str(change['new']) == "--Select a video--":
            selected_video_name = str(change['new'])
            video_path = os.path.join(self.data_path, self.selected_folder, self.selected_video)
            if(self.toDisplay):
              video_player = Video.from_file(video_path)
              display(video_player)

    def get_Selected_Folder(self):
        return self.selected_folder

    def get_Selected_Video(self):
        return self.selected_video

    def get_Full_Video_Path(self):
        return os.path.join(self.data_path, self.selected_folder, self.selected_video)


In [ ]:
dropdown = FolderDropdown(data_path, toDisplayVideo=True)
dropdown.display()

In [ ]:
#@title Get selected Video
selected_Video = dropdown.get_Full_Video_Path()
print(selected_Video)

/content/FollowYourPose/data/00MFE/00MFE.mp4


# **Inference Section**

In [ ]:
#@title Edit Config File

#@markdown Setup Config file for inference
from omegaconf import OmegaConf

CONFIG_NAME = "configs/pose_sample.yaml" #@param {type:"string"}

prompts = "Iron man on the beach" #@param {type:"string"}
video_length = 24 #@param {type:"number"}
width = 512 #@param {type:"number"}
height = 512 #@param {type:"number"}

config = {
  "pretrained_model_path": "./checkpoints/stable-diffusion-v1-4",
  "output_dir": "output",
  "validation_data": {
    "prompts": [
      prompts
    ],
    "video_length": video_length,
    "width": width,
    "height": height,
    "num_inference_steps": 50,
    "guidance_scale": 12.5,
    "use_inv_latent": False,
    "num_inv_steps": 50,
    "dataset_set": "val"
  },
  "train_batch_size": 1,
  "validation_steps": 100,
  "resume_from_checkpoint": "./checkpoints/followyourpose_checkpoint-1000",
  "seed": 33,
  "mixed_precision": "no",
  "gradient_checkpointing": False,
  "enable_xformers_memory_efficient_attention": True
}

OmegaConf.save(config, CONFIG_NAME)


In [ ]:
#@title Running Inference

!pwd
!TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch txt2video.py --config="configs/pose_sample.yaml"  --skeleton_path="./pose_example/vis_ikun_pose1.MOV"


/content/FollowYourPose
2023-10-14 15:40:43.350269: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-14 15:40:46.205251: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `0`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2023-10-14 15:40:57.975681: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
I0000 00:00:1697298061.939081    

# **MM Pose Section**

In [ ]:
#@title Load API
%cd /content/FollowYourPose/
from inference_followyourpose import *
config_runner = merge_config_then_run()

/content/FollowYourPose
Fetching Space from: https://huggingface.co/spaces/YueMafighting/mmpose-estimation
Loaded as API: https://yuemafighting-mmpose-estimation.hf.space ✔


In [ ]:
#@title edit parameters

#Select the original video
dropdown = FolderDropdown(data_path, toDisplayVideo=True)
dropdown.display()

Dropdown(description='Select a folder:', options=('--Select a folder--', 'X8AP2'), value='--Select a folder--'…

Dropdown(description='Select a video:', options=('--Select a video--',), value='--Select a video--')

Output()

In [ ]:
#Get the full path, folder name and video name from the FolderDropdown object
#(Run this after dropdown selection)
video_path=dropdown.get_Full_Video_Path()
folder_path = dropdown.get_Selected_Folder()
video_name = dropdown.get_Selected_Video()
print(video_path)

/content/FollowYourPose/data/X8AP2/X8AP2.mp4


In [ ]:
#@title Edit Config file (configs/pose_sample.yaml)


#@title Edit Config File

#@markdown Setup Config file for inference
from omegaconf import OmegaConf

CONFIG_NAME = "configs/pose_sample.yaml" #@param {type:"string"}

pretrained_model_path = "./checkpoints/stable-diffusion-v1-4" #@param {type:"string"}
output_dir = "outputSkeleton" #@param {type:"string"}
prompts = "Iron man on the beach" #@param {type:"string"}
video_length = 32 #@param {type:"number"}
width = 512 #@param {type:"number"}
height = 512 #@param {type:"number"}
num_inference_steps = 50 #@param {type:"number"}
guidance_scale = 12.5 #@param {type:"number"}
num_inv_steps = 50 #@param {type:"number"}
train_batch_size = 1 #@param {type:"number"}
validation_steps = 100 #@param {type:"number"}
resume_from_checkpoint = "./checkpoints/followyourpose_checkpoint-1000" #@param {type:"string"}


config = {
  "pretrained_model_path": pretrained_model_path,
  "output_dir": output_dir,
  "validation_data": {
    "prompts": [
      prompts
    ],
    "video_length": video_length,
    "width": width,
    "height": height,
    "num_inference_steps": num_inference_steps,
    "guidance_scale": guidance_scale,
    "use_inv_latent": False,
    "num_inv_steps": num_inv_steps,
    "dataset_set": "val"
  },
  "train_batch_size": train_batch_size,
  "validation_steps": validation_steps,
  "resume_from_checkpoint": "./checkpoints/followyourpose_checkpoint-1000",
  "seed": 33,
  "mixed_precision": "no",
  "gradient_checkpointing": False,
  "enable_xformers_memory_efficient_attention": True
}

OmegaConf.save(config, CONFIG_NAME)





In [ ]:
#Define function parameters
target_prompt="A person is eating, then opening and closing a book, then using a phone and finally taking medicine in a living room."#@param {type: "string"}
num_steps=10 #@param {type: "number"}
guidance_scale=12.5 #@param {type: "number"}
video_type="Raw Video" #@param {type: "string"}
user_input_video=None
start_sample_frame=0 #@param {type: "number"}
n_sample_frame=10 #@param {type: "number"}
stride=1 #@param {type: "number"}
left_crop=0 #@param {type: "number"}
right_crop=0 #@param {type: "number"}
top_crop=0 #@param {type: "number"}
bottom_crop=0 #@param {type: "number"}

In [ ]:
#@title Generate Skeleton
%cd $home_path
output = config_runner.run(video_path, target_prompt, num_steps, guidance_scale, video_type, user_input_video, start_sample_frame, n_sample_frame, stride, left_crop, right_crop, top_crop, bottom_crop)


import shutil
rename_name = video_name.split(".")[0] + ".mp4"
#path to the new location
renamed_path = os.path.join("./", output_dir, folder_path, rename_name)
#Rename current output to input name
os.rename(output, rename_name)
#Make Directory
os.makedirs(os.path.join("./", output_dir, dropdown.get_Selected_Folder()), exist_ok=True)
#Move output video to new directory
shutil.move(rename_name, renamed_path)

# **Training Section**

In [ ]:
#@title Add CharadesVideo.json

%%writefile $data_path/CharadesVideo.json
[
]


Overwriting /content/FollowYourPose/data/CharadesVideo.json


In [ ]:
#@title Add video properties into CharadesVideo.json
import ipywidgets as widgets
from IPython.display import display
import json

%cd $home_path

# Define the text fields
id_field = widgets.Text(description='video name:')
scripts_field = widgets.Text(description='script:')

# Define help text using Label widgets
id_help = widgets.Label(value='Enter the video name (Without .mp4)')
scripts_help = widgets.Label(value='Enter a short description of the video')

# Define a button
button = widgets.Button(description='Submit')

# Define a function to be executed on button click
def on_button_click(b):
    id = id_field.value
    scripts = scripts_field.value

    # Load existing JSON data
    with open('./data/CharadesVideo.json', 'r') as json_file:
        data = json.load(json_file)

    # Create a new JSON object
    new_data = {
        "id": id,
        "script": scripts
    }

    # Append the new object to the existing JSON list
    data.append(new_data)

    # Write the updated JSON data back to the file
    with open('./data/CharadesVideo.json', 'w') as json_file:
        json.dump(data, json_file, indent=4)
    print(f'Successfully Added Video ID: {id}')

# Attach the function to the button's click event
button.on_click(on_button_click)

# Display the widgets
display(id_help, id_field, scripts_help, scripts_field, button)

/content/FollowYourPose


Label(value='Enter the video name (Without .mp4)')

Text(value='', description='video name:')

Label(value='Enter a short description of the video')

Text(value='', description='script:')

Button(description='Submit', style=ButtonStyle())

Successfully Added Video ID: X8AP2
Successfully Added Video ID: X8AP2
Successfully Added Video ID: X8AP2
Successfully Added Video ID: X8AP2


In [ ]:
#@title Remove video from CharadesVideo.json (Optional)
%cd $home_path

import ipywidgets as widgets
from IPython.display import display
import json

# Create a function to display IDs and remove on button click
def display_and_remove_id():
    # Load existing JSON data
    json_file_path = './data/CharadesVideo.json'
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)

    # Define a dropdown widget to display IDs
    id_dropdown = widgets.Dropdown(options=[item["id"] for item in data], description='Select ID:')

    # Define a button to remove the selected ID
    remove_button = widgets.Button(description='Remove ID')

    # Define an output widget for displaying success messages
    success_output = widgets.Output()

    # Define a function to remove the selected ID and display a success message
    def on_remove_button_click(b):
        selected_id = id_dropdown.value
        data[:] = [item for item in data if item["id"] != selected_id]
        id_dropdown.options = [item["id"] for item in data]

        # Save the updated data to the JSON file
        with open(json_file_path, 'w') as json_file:
            json.dump(data, json_file, indent=4)

        # Display a success message
        with success_output:
            print(f'Successfully removed ID: {selected_id}')

    # Attach the remove function to the remove button's click event
    remove_button.on_click(on_remove_button_click)

    # Display the widgets
    display(id_dropdown, remove_button, success_output)

# Call the function to display IDs and the remove button
display_and_remove_id()



/content/FollowYourPose


Dropdown(description='Select ID:', options=('X8AP2',), value='X8AP2')

Button(description='Remove ID', style=ButtonStyle())

Output()

In [ ]:
#@title Edit Config File
%cd $home_path

#@markdown Setup Config file for training

from omegaconf import OmegaConf
#@markdown Config File Path
CONFIG_NAME = "configs/pose_train.yaml"     #@param {type:"string"}


#@markdown Pretrained Model  Path
pretrained_model_path = "./checkpoints/stable-diffusion-v1-4" #@param {type:"string"}

#@markdown model Output Path
#Location and name of newly trained model
output_dir = "output/model2"                        #@param {type:"string"}

#@markdown train_data
# Training video subfolder
train_video_path = "./data"                         #@param {type:"string"}
train_prompt = "A person is eating, then opening and closing a book, then using a phone and finally taking medicine in a living room."                    #@param {type:"string"}
n_sample_frames = 4                                 #@param {type:"number"}
# width = 512                                       #@param {type:"number"}
# height = 512                                      #@param {type:"number"}

learning_rate = 3e-5                                #@param {type:"number"}
#This might help determin no of epochs (refer train_followyourpose.py)
train_batch_size = 1                                #@param {type:"number"}
# might need to change the training code to take in batch size from config file
max_train_steps = 50                                #@param {type:"number"}
checkpointing_steps = 25                            #@param {type:"number"}
validation_steps = 50                              #@param {type:"number"}

skeleton_path = "./outputSkeleton/X8AP2/X8AP2.mp4"  #@param {type:"string"}

config = {
  "pretrained_model_path": "./checkpoints/stable-diffusion-v1-4",
  "output_dir": output_dir,
  "train_data": {
    "video_path": train_video_path,
    "prompt": train_prompt,
    "n_sample_frames": n_sample_frames,
    "width": 512,
    "height": 512,
    "sample_start_idx": 0,
    "sample_frame_rate": 4,
    "dataset_set": "train"
  },
  "validation_data": {
    "prompts": [
      "A Iron man on the beach",
      "A Spider man on the snow",
      "A Superman on the street",
      "A boy on the forest"
    ],
    "video_length": 1,
    "width": 512,
    "height": 512,
    "num_inference_steps": 1,
    "guidance_scale": 12.5,
    "use_inv_latent": False,
    "num_inv_steps": 1,
    "dataset_set": "val"
  },
  "learning_rate": learning_rate,
  "train_batch_size": train_batch_size,
  "max_train_steps": max_train_steps,
  "checkpointing_steps": checkpointing_steps,
  "validation_steps": validation_steps,
  "trainable_modules": [
    "attn1.to_q",
    "attn2.to_q",
    "attn_temp",
    "conv_temporal"
  ],
  "skeleton_path": skeleton_path,
  "seed": 33,
  "mixed_precision": "no",
  "use_8bit_adam": False,
  "gradient_checkpointing": False,
  "enable_xformers_memory_efficient_attention": True
}

OmegaConf.save(config, CONFIG_NAME)

/content/FollowYourPose


## Start Training

In [ ]:
!TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch train_followyourpose.py --config=CONFIG_NAME

2023-10-25 02:18:34.106294: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-25 02:18:34.106361: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-25 02:18:34.106400: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-25 02:18:36.572851: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dyna

# **Monitor Progress**
Displayed the progress of the model training using progress bars

In [ ]:
!pip install tqdm

In [ ]:
# Method 1: Read the epoch size from edit config file

from tqdm import tqdm
import subprocess
from omegaconf import OmegaConf

# Define the training script and configuration file
training_script = "train_followyourpose.py"
config_file = "configs/pose_train_testing.yaml"  # Use the updated config file name

# Get the number of training steps from the config
config = OmegaConf.load(CONFIG_NAME)
train_steps = config.max_train_steps

# Define ANSI escape codes for blue color
blue_color = "\033[94m"  # Blue text color
end_color = "\033[0m"    # Reset to default color

# Create a colored progress bar with blue bars and custom formatting
bar_format = f"Training Progress: |{blue_color}{{bar}}{end_color}| {{percentage:3.0f}}% {{n_fmt}}/{{total_fmt}}"
with tqdm(total=train_steps, bar_format=bar_format, ncols=100, dynamic_ncols=True) as pbar:
    for step in range(train_steps):
        # Run the training script
        command = f"TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch {training_script} --config={config_file}"
        subprocess.run(command, shell=True)

        # Update the progress bar
        pbar.update(1)


Training Progress: |██████████| 100% 3/3


In [ ]:
# Method 2: Ownself set fixed size to run

from tqdm import tqdm
import subprocess
import time

# Define the training script and configuration file
training_script = "train_followyourpose.py"
config_file = "configs/pose_train.yaml"

# Define the total number of epochs
total_epochs = 3  # Set the number of training epochs

# Initialize the progress bar
with tqdm(total=total_epochs, desc="Training Progress") as pbar:
    for epoch in range(total_epochs):
        # Run the training script
        command = f"TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch {training_script} --config={config_file}"
        subprocess.run(command, shell=True)

        # Update the progress bar
        pbar.update(1)
        time.sleep(1)  # Simulate progress, replace with your actual training logic


Training Progress: 100%|██████████| 3/3 [03:30<00:00, 70.15s/it]


## **Fine Tuning**

In [ ]:
import yaml
import ipywidgets as widgets
import os
from IPython.display import display
from ipywidgets import Button, VBox
from google.colab import files
from omegaconf import OmegaConf

# Load the config file
config = OmegaConf.load(CONFIG_NAME)

# Get the absolute path of the config file
config_file_path = os.path.abspath(CONFIG_NAME)

# Read the YAML file
with open(config_file_path, 'r') as file:
    config_data = yaml.safe_load(file)

# Extract "skeleton path" value
skeleton_path_file = os.path.abspath(config_data['skeleton_path'])
skeleton_path_directory = os.path.dirname(skeleton_path_file)
skeleton_file_name = os.path.basename(skeleton_path_file)

# Extract the "prompt" values from the YAML data
prompts = config_data['validation_data']['prompts']

# Output widget to display the imported video filename
output_widget = widgets.Output()

# List to keep track of imported video strings
imported_video_strings = []

# Create "Upload Video" button
upload_button = Button(description="Upload Video")
upload_button.on_click(on_upload_button_click)  # Set up the event handler here

# Create "Replace Video" button
replace_button = Button(description="Replace Video", disabled=True)
replace_button.on_click(on_replace_button_click)  # Set up the event handler here

# Create a VBox to stack the widgets vertically
vbox = VBox([upload_button, output_widget, replace_button])

# Function to handle dropdown changes
def on_dropdown_change(change):
    selected_prompt = change.new
    # You can perform any additional actions here if needed
    print(f"Selected prompt: {selected_prompt}")

    # Display the existing widgets
    display(vbox)

# Observer for prompt_dropdown
prompt_dropdown = widgets.Dropdown(
    options=prompts,
    description='Select Prompt:',
)
prompt_dropdown.observe(on_dropdown_change, names='value')

# Display the initial widgets
display(prompt_dropdown)
display(vbox)

# Define a function to handle video import
def on_upload_button_click(b):
    # Prompt the user to upload a file
    uploaded = files.upload()

    if uploaded:
        uploaded_filename = next(iter(uploaded))
        imported_filename_output = f"Imported Video: {uploaded_filename}"

        with output_widget:
            output_widget.clear_output(wait=True)
            imported_video_strings.append(imported_filename_output)
            # Display all imported video strings
            for video_string in imported_video_strings:
                print(video_string)

        # Store the uploaded content in a global variable
        global uploaded_content
        uploaded_content = uploaded[uploaded_filename]

        replace_button.disabled = False  # Enable the replace button after importing a new video

# Define a function to handle video replacement
def on_replace_button_click(b):
    # Use the global variable to access the uploaded content
    uploaded_file_content = uploaded_content

    # Save the uploaded file to the specified path, replacing the existing video with the same name
    replace_video_path = os.path.join(skeleton_path_directory, skeleton_file_name)
    with open(replace_video_path, 'wb') as f:
        f.write(uploaded_file_content)

    # You can perform any additional actions here if needed
    print(f"Video {skeleton_file_name} replaced successfully.")

    # Reset the imported video strings
    imported_video_strings.clear()


## **Select Subfolder from Data folder**

In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# Path to the 'data' folder
data_folder_path = '/content/FollowYourPose/data'

# List the subdirectories (subfolders) in the 'data' folder
subfolders = [subfolder for subfolder in os.listdir(data_folder_path) if os.path.isdir(os.path.join(data_folder_path, subfolder))]

# Create a dropdown widget
subfolder_dropdown = widgets.Dropdown(
    options=[''] + subfolders,  # Include an empty option to represent "nothing selected"
    description='Select Subfolder:'
)

output_widget = widgets.Output()

# Initialize selected_folder_path as a global variable
selected_folder_path = None

# Define a function to handle dropdown changes
def on_dropdown_change(change):
    global selected_folder_path  # Make selected_folder_path a global variable
    selected_subfolder = change.new
    with output_widget:
        output_widget.clear_output()
        if selected_subfolder:
            # Set selected_folder_path when a subfolder is selected
            selected_folder_path = os.path.join(data_folder_path, selected_subfolder)
            print(f"Selected Subfolder: {selected_subfolder}")
            print(f"Full Path: {selected_folder_path}")
        else:
            # Clear selected_folder_path when no subfolder is selected
            selected_folder_path = None
            print("No subfolder selected")

# Observe changes in the dropdown list
subfolder_dropdown.observe(on_dropdown_change, names='value')

# Display the initial widgets
display(subfolder_dropdown)
display(output_widget)

# Now, selected_folder_path is a global variable accessible outside the code block
# If nothing is selected, selected_folder_path will be None

In [ ]:
print(selected_folder_path)

# **New section**

## Others

# **Addition of Caption**

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import os
import configparser
import cv2

video_name_input = widgets.Text(
    value='',
    placeholder='Enter video name',
    description='Video name:'
)

caption_input = widgets.Text(
    value='',
    placeholder='Enter caption',
    description='Caption:'
)

save_button = widgets.Button(
    description="Save Video and Caption"
)

output = widgets.Output()


def save_video(video_name):
    # Define the output path for the video file
    video_output_path = f'/content/FollowYourPose/{video_name}.mp4'

    # Open the input video file (you should provide the path to your input video)
    input_video_path = video_path
    cap = cv2.VideoCapture(input_video_path)

    # Get video properties
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    frame_rate = int(cap.get(5))

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 format
    out = cv2.VideoWriter(video_output_path, fourcc, frame_rate, (frame_width, frame_height))

    # Read frames from the input video and write them to the output video
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        out.write(frame)

    # Release video objects
    cap.release()
    out.release()

def save_caption_to_config(caption_text, config_file_path):
    config = configparser.ConfigParser()
    config.add_section('Captions')
    config.set('Captions', 'caption_text', caption_text)
    with open(config_file_path, 'w') as configfile:
        config.write(configfile)

def save_video_and_caption(_):
    video_name = video_name_input.value
    caption_text = caption_input.value

    # Define the output path for the INI file
    config_file_path = f'/content/FollowYourPose/{video_name}.ini'

    # Save the caption to the INI file
    save_caption_to_config(caption_text, config_file_path)

    # Save the video in .mp4 format
    save_video(video_name)

    # Display a message in the output widget to indicate that both the video and caption have been saved
    with output:
        print(f"Saved video to file: {video_name}.mp4")
        print(f"Saved caption to INI file: {config_file_path}")

save_button.on_click(save_video_and_caption)

widget_container = widgets.VBox([video_name_input, caption_input, save_button, output])

display(widget_container)


In [ ]:
import os
import subprocess
from IPython.display import display, Video
import configparser

# Specify the folder where the videos with captions are saved
output_folder = "/content/FollowYourPose/"

# List all video files in the output folder
video_files = [file for file in os.listdir(output_folder) if file.endswith(".mp4")]

# Specify the fixed video size (e.g., 300x300)
fixed_video_width = 300
fixed_video_height = 300

# Specify the fixed frame size (e.g., 350x400)
fixed_frame_width = 350
fixed_frame_height = 400

# Specify the caption offset (e.g., 50 pixels above the bottom)
caption_offset = 50

# Iterate through the video files
for video_file in video_files:
    input_video_path = os.path.join(output_folder, video_file)
    output_video_path = os.path.join(output_folder, f"{video_file.split('.')[0]}_with_caption.mp4")

    # Read the config file
    config_file_path = os.path.join(output_folder, f"{video_file.split('.')[0]}.ini")
    if os.path.exists(config_file_path):
        config = configparser.ConfigParser()
        config.read(config_file_path)
        caption_text = config.get("Captions", "caption_text")
    else:
        caption_text = "No caption available"

    # Use FFmpeg to resize the video to the fixed dimensions and overlay the caption text at the bottom
    ffmpeg_command = (
        "ffmpeg -i {input_video} -vf "
        "\"scale={video_width}:{video_height},"
        "drawtext=text='{caption}':x=10:y=h-{caption_offset}:fontsize=24:fontcolor=white,"
        "scale={frame_width}:{frame_height}\" "
        "-c:v libx264 -preset medium -crf 23 -c:a aac -strict experimental -b:a 192k {output_video}"
    )

    subprocess.run(
        ffmpeg_command.format(
            input_video=input_video_path,
            caption=caption_text,
            video_width=fixed_video_width,
            video_height=fixed_video_height,
            caption_offset=caption_offset,
            frame_width=fixed_frame_width,
            frame_height=fixed_frame_height,
            output_video=output_video_path,
        ),
        shell=True,
    )

    # Display and play the modified video
    display(Video(output_video_path, embed=True, width=fixed_frame_width, height=fixed_frame_height))

    print(f"Processed {video_file} with caption from config file.")


Processed hello.mp4 with caption from config file.


## **Select and save the best-trained model and delete other models generated during training**

In [ ]:
import os
from ipywidgets import Dropdown, Button, Output

# Step 1: List saved models
def list_saved_models():
    # List saved models in the current directory
    folder = "/content/FollowYourPose/output/"
    saved_models = [f for f in os.listdir(folder) if os.path.isdir(os.path.join(folder, f))]
    return saved_models

# Step 2: User selection
def select_best_model(available_models):
    print("Available Models:")
    for i, model in enumerate(available_models):
        print(f"{i + 1}. {model}")
    selection = int(input("Select the best model (enter the corresponding number): "))
    if 1 <= selection <= len(available_models):
        return available_models[selection - 1]
    else:
        print("Invalid selection. Please try again.")
        return select_best_model(available_models)

# Step 3: Deletion of unused models
def delete_unused_models(user_selected_model, available_models):
    print(f"Deleting unused models (keeping {user_selected_model})")
    for model in available_models:
        if model != user_selected_model:
            os.remove(model)
            print(f"Deleted: {model}")

# Step 4: UI Feedback
out = Output()

# Function to handle best model selection and deletion
def handle_best_model_selection(b):
    with out:
        available_models = list_saved_models()
        if not available_models:
            print("No models found.")
            return

        user_selected_model = select_best_model(available_models)
        delete_unused_models(user_selected_model, available_models)
        print(f"The best model is: {user_selected_model}")
        print("Other models have been deleted.")

# UI components
dropdown = Dropdown(
    options=list_saved_models(),
    description='Select Model:'
)
button = Button(description="Select Best Model and Delete Others")
button.on_click(handle_best_model_selection)

# Display UI components
display(dropdown, button, out)


Dropdown(description='Select Model:', options=('.ipynb_checkpoints', 'model2', 'model1'), value='.ipynb_checkp…

Button(description='Select Best Model and Delete Others', style=ButtonStyle())

Output()